<a href="https://colab.research.google.com/github/jenskuehne-cmd/Colab_Skripte_OEE/blob/main/Notebooks/RoleMappingAspire/MasterRMList_noaspireRolesNeeded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dieses Notebook dient dazu, den Status 'No Aspire Role needed' (Keine Aspire Rolle benötigt) für Benutzer zwischen zwei Google Sheets zu synchronisieren.

**Was das Notebook genau macht:**

1.  **Authentifizierung:** Es meldet sich bei Ihrem Google-Konto an, um sicherzustellen, dass es auf Ihre Google Sheets zugreifen darf.
2.  **Daten laden (Quelle):** Es liest Daten aus einem Google Sheet namens 'KAU Daten' (speziell aus dem Tab 'RM_KAU'). Hier wird nach der Spalte 'UserID' und 'No Aspire Role needed' gesucht.
3.  **Daten laden (Ziel):** Es liest Daten aus einem anderen Google Sheet namens 'RoleMap' (speziell aus dem Tab 'RoleMap'). Auch hier werden die Spalten 'UserID' und 'No Aspire Role needed' betrachtet.
4.  **Vergleich und Aktualisierung:**
    *   Das Notebook identifiziert alle 'UserIDs' im 'KAU Daten'-Sheet, bei denen die Spalte 'No Aspire Role needed' auf 'TRUE' gesetzt ist.
    *   Anschließend geht es das 'RoleMap'-Sheet durch und setzt für alle diese identifizierten 'UserIDs' den Wert in der Spalte 'No Aspire Role needed' ebenfalls auf 'TRUE'.
5.  **Speichern:** Die aktualisierten Daten werden dann zurück in das 'RoleMap'-Sheet geschrieben, wobei die vorhandenen Kopfzeilen erhalten bleiben.

**Kurz gesagt:** Wenn ein Benutzer im 'KAU Daten'-Sheet als jemand markiert ist, der keine Aspire Rolle benötigt, stellt dieses Skript sicher, dass diese Markierung auch im 'RoleMap'-Sheet korrekt übernommen wird.

In [ ]:
import gspread
import pandas as pd
from google.colab import auth
from google.auth import default

# Authentifizierung für Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Source Google Sheet (KAU Daten)
source_sheet_url = "https://docs.google.com/spreadsheets/d/1L37Bx2RKAH9I5OP3v-jG2LDjGzi3mdbiFBuz9FIflKM/edit"
source_sheet = gc.open_by_url(source_sheet_url)
source_worksheet = source_sheet.worksheet("RM_KAU")

# Target Google Sheet (RoleMap)
target_sheet_url = "https://docs.google.com/spreadsheets/d/1MGlcO6cReCiouwZ9k8caSD5GS3TQGgczYtGD8rm5wAU/edit"
target_sheet = gc.open_by_url(target_sheet_url)
target_worksheet = target_sheet.worksheet("RoleMap")

# Daten aus Source-Tabelle abrufen
source_data = source_worksheet.get_all_values()
source_df = pd.DataFrame(source_data[1:], columns=source_data[0])  # Erste Zeile als Header nutzen

# Daten aus Target-Tabelle abrufen
target_data = target_worksheet.get_all_values()
target_df = pd.DataFrame(target_data[2:], columns=target_data[1])  # Zweite Zeile als Header nutzen

# Sicherstellen, dass die relevanten Spalten existieren
source_column_userid = "UserID"
source_column_flag = "No Aspire Role needed"
target_column_flag = "No Aspire Role needed"

if source_column_userid not in source_df.columns or source_column_flag not in source_df.columns:
    raise ValueError("Spaltennamen in der Source-Tabelle sind nicht korrekt!")

if source_column_userid not in target_df.columns or target_column_flag not in target_df.columns:
    raise ValueError("Spaltennamen in der Target-Tabelle sind nicht korrekt!")

# Liste der UserIDs mit "TRUE" in "No Aspire Role needed"
true_user_ids = set(source_df[source_df[source_column_flag] == "TRUE"][source_column_userid])

# "No Aspire Role needed" in der Target-Tabelle auf TRUE setzen
target_df.loc[target_df[source_column_userid].isin(true_user_ids), target_column_flag] = "TRUE"

# Aktualisierte Werte zurück in die Target-Tabelle schreiben (unter Beibehaltung der leeren Zeile)
updated_values = [target_data[0]] + [target_data[1]] + target_df.values.tolist()
target_worksheet.update(updated_values)

print("Update abgeschlossen! Die Spalte 'No Aspire Role needed' wurde in der Target-Tabelle aktualisiert.")
